In [ ]:
import sqlite3
import pandas as pd

In [ ]:
csv_file_path = '/content/Corona Virus Dataset.csv'  # Update with your file path
df = pd.read_csv(csv_file_path)
df.head()  # Display the first few rows to verify the data


,Province,Country/Region,Latitude,Longitude,Date,Confirmed,Deaths,Recovered
0,Afghanistan,Afghanistan,33.93911,67.709953,22-01-2020,0,0,0
1,Afghanistan,Afghanistan,33.93911,67.709953,23-01-2020,0,0,0
2,Afghanistan,Afghanistan,33.93911,67.709953,24-01-2020,0,0,0
3,Afghanistan,Afghanistan,33.93911,67.709953,25-01-2020,0,0,0
4,Afghanistan,Afghanistan,33.93911,67.709953,26-01-2020,0,0,0


In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y').dt.strftime('%Y-%m-%d')

In [ ]:
conn = sqlite3.connect('corona_virus_analysis.db')
df.to_sql('corona_data', conn, if_exists='replace', index=False)

78386

In [ ]:
query = 'SELECT * FROM corona_data LIMIT 5'
result = pd.read_sql(query, conn)
print(result)

      Province Country/Region  Latitude  Longitude        Date  Confirmed  \
0  Afghanistan    Afghanistan  33.93911  67.709953  2020-01-22          0   
1  Afghanistan    Afghanistan  33.93911  67.709953  2020-01-23          0   
2  Afghanistan    Afghanistan  33.93911  67.709953  2020-01-24          0   
3  Afghanistan    Afghanistan  33.93911  67.709953  2020-01-25          0   
4  Afghanistan    Afghanistan  33.93911  67.709953  2020-01-26          0   

   Deaths  Recovered  
0       0          0  
1       0          0  
2       0          0  
3       0          0  
4       0          0  


In [ ]:
# To avoid any errors, check missing value / null value
#Q1. Write a code to check NULL values
#Q2. If NULL values are present, update them with zeros for all columns.
#Q3. check total number of rows
#Q4. Check what is start_date and end_date
#Q5. Number of month present in dataset
#Q6. Find monthly average for confirmed, deaths, recovered
#Q7. Find most frequent value for confirmed, deaths, recovered each month
#Q8. Find minimum values for confirmed, deaths, recovered per year
#Q9. Find maximum values of confirmed, deaths, recovered per year
#Q10. The total number of case of confirmed, deaths, recovered each month
#Q11. Identify the top 10 countries/regions with the highest number of confirmed cases.
#Q12. Calculate the daily average number of new confirmed cases globally.
#Q13. Determine the mortality rate (percentage) for each country/region.
#Q14. Find Country having highest number of the Confirmed case
#Q15. Find Country having lowest number of the death case
#Q16. Find top 5 countries having highest recovered case

In [ ]:
#Q1. Write a code to check NULL values
query = '''
SELECT
    SUM(CASE WHEN Province IS NULL THEN 1 ELSE 0 END) AS Null_Province,
    SUM(CASE WHEN "Country/Region" IS NULL THEN 1 ELSE 0 END) AS Null_Country_Region,
    SUM(CASE WHEN Latitude IS NULL THEN 1 ELSE 0 END) AS Null_Latitude,
    SUM(CASE WHEN Longitude IS NULL THEN 1 ELSE 0 END) AS Null_Longitude,
    SUM(CASE WHEN Date IS NULL THEN 1 ELSE 0 END) AS Null_Date,
    SUM(CASE WHEN Confirmed IS NULL THEN 1 ELSE 0 END) AS Null_Confirmed,
    SUM(CASE WHEN Deaths IS NULL THEN 1 ELSE 0 END) AS Null_Deaths,
    SUM(CASE WHEN Recovered IS NULL THEN 1 ELSE 0 END) AS Null_Recovered
FROM Corona_Data;
'''
null_values = pd.read_sql(query, conn)
null_values.head()

,Null_Province,Null_Country_Region,Null_Latitude,Null_Longitude,Null_Date,Null_Confirmed,Null_Deaths,Null_Recovered
0,0,0,0,0,0,0,0,0


In [ ]:
#Q2. If NULL values are present, update them with zeros for all columns.

#There is no NULL values in the given dataset. so no need to update

In [ ]:
#Q3. check total number of rows
query = '''SELECT COUNT(*) AS Total_Rows
FROM corona_data;
'''
total_rows = pd.read_sql(query, conn)
total_rows.head()

,Total_Rows
0,78386


In [ ]:
#Q4. Check what is start_date and end_date
query = '''SELECT MIN(Date) AS start_date, MAX(Date) AS end_date FROM corona_data;'''
date_range = pd.read_sql(query, conn)
date_range.head()


,start_date,end_date
0,2020-01-22,2021-06-13


In [ ]:
#Q5. Number of month present in dataset
query = '''SELECT COUNT(DISTINCT strftime(Date, '%Y-%m')) AS Num_Months
FROM corona_data;
'''
num_months = pd.read_sql(query, conn)
num_months.head()


,Num_Months
0,0


In [11]:
#Q6. Find monthly average for confirmed, deaths, recovered
query = '''SELECT strftime('%m',Date) AS MONTH,
	   ROUND(AVG(Confirmed),2) AS Avg_Confirmed,
	   ROUND(AVG(Deaths),2) AS Avg_Deaths,
	   ROUND(AVG(Recovered),2) AS Avg_Recovered
FROM corona_data
GROUP BY strftime('%m',Date);'''
Avg = pd.read_sql(query, conn)
Avg.head()

,MONTH,Avg_Confirmed,Avg_Deaths,Avg_Recovered
0,01,2958.28,63.68,1451.46
1,02,1203.12,34.28,769.10
2,03,1538.96,33.93,840.08
3,04,2602.58,59.98,1623.21
4,05,2290.05,53.53,2162.90


In [12]:
#Q7. Find most frequent value for confirmed, deaths, recovered each month
query = '''SELECT Month, Confirmed, Frequency
FROM (
    SELECT strftime('%m', Date) AS Month,
           Confirmed,
           COUNT(*) AS Frequency,
           ROW_NUMBER() OVER(PARTITION BY strftime('%m', Date) ORDER BY COUNT(*) DESC) AS RowNum
    FROM corona_data
    GROUP BY strftime('%m', Date), Confirmed
) AS T
WHERE RowNum = 1;
'''
most_frequent = pd.read_sql(query, conn)
most_frequent.head()


,Month,Confirmed,Frequency
0,01,0,2573
1,02,0,5073
2,03,0,3386
3,04,0,2501
4,05,0,2901


In [13]:
#Q8. Find minimum values for confirmed, deaths, recovered per year
query = ''' SELECT SUBSTR(Date, 1, 4) AS Year,
  MIN(Confirmed) AS Min_Confirmed,
  MIN(Deaths) AS Min_Deaths,
  MIN(Recovered) AS Min_Recovered
  FROM corona_data
  GROUP BY Year;
  '''
min_year = pd.read_sql(query, conn)
min_year.head()

,Year,Min_Confirmed,Min_Deaths,Min_Recovered
0,2020,0,0,0
1,2021,0,0,0


In [14]:
#Q9. Find maximum values of confirmed, deaths, recovered per year
query = ''' SELECT SUBSTR(Date, 1, 4) AS Year,
  MAX(Confirmed) AS Max_Confirmed,
  MAX(Deaths) AS Max_Deaths,
  MAX(Recovered) AS Max_Recovered
  FROM corona_data
  GROUP BY Year;
  '''
max_year = pd.read_sql(query, conn)
max_year.head()

,Year,Max_Confirmed,Max_Deaths,Max_Recovered
0,2020,823225,3752,1123456
1,2021,414188,7374,422436


In [15]:
#Q10. The total number of case of confirmed, deaths, recovered each month
query = '''SELECT strftime('%m',Date) AS Month,
       SUM(Confirmed) AS Total_Confirmed,
       SUM(Deaths) AS Total_Deaths,
       SUM(Recovered) AS Total_Recovered
FROM corona_data
GROUP BY Month;'''
total_case = pd.read_sql(query, conn)
total_case.head()

,Month,Total_Confirmed,Total_Deaths,Total_Recovered
0,01,18678589,402083,9164490
1,02,10560976,300890,6751190
2,03,14694026,323966,8021083
3,04,24047819,554220,14998494
4,05,21865416,511110,20651389


In [17]:
#Q11. Identify the top 10 countries/regions with the highest number of confirmed cases.
query = '''SELECT "Country/Region", SUM(Confirmed) AS Total_Confirmed
FROM corona_data
GROUP BY "Country/Region"
ORDER BY Total_Confirmed DESC
LIMIT 10;'''
top_10_countries = pd.read_sql(query, conn)
top_10_countries.head()

,Country/Region,Total_Confirmed
0,US,33461982
1,India,29460523
2,Brazil,17412766
3,France,6106009
4,Turkey,5330447


In [18]:
#Q12. Calculate the daily average number of new confirmed cases globally.
query = '''SELECT ROUND(AVG(Daily_cases),2) AS Avg_Daily_newCases
FROM (
    SELECT Date, SUM(Confirmed) AS Daily_cases
	FROM corona_data
	GROUP BY Date
)AS global_daily_cases;
  '''
avg_new_confirmed = pd.read_sql(query, conn)
avg_new_confirmed.head()

,Avg_Daily_newCases
0,332151.56


In [21]:
#Q13. Determine the mortality rate (percentage) for each country/region.
query = '''SELECT "Country/Region",
      SUM(Deaths) AS Total_Deaths,
      sum(Confirmed) AS Total_Confirmed,
      ROUND((cast(sum(Deaths)  AS FLOAT)/ sum(Confirmed)) * 100, 2) AS Mortality_Rate
FROM corona_data
GROUP BY "Country/Region";'''
mortality_rate = pd.read_sql(query, conn)
mortality_rate.head()

,Country/Region,Total_Deaths,Total_Confirmed,Mortality_Rate
0,Afghanistan,3527,89861,3.92
1,Algeria,3571,133388,2.68
2,Argentina,85343,4124190,2.07
3,Australia,911,30365,3.00
4,Austria,10663,648620,1.64


In [22]:
#Q14. Find Country having highest number of the Confirmed case
query = ''' SELECT "Country/Region", MAX(Confirmed) AS Max_Confirmed
FROM corona_data;
'''
Confirmed_cases = pd.read_sql(query, conn)
Confirmed_cases.head()


,Country/Region,Max_Confirmed
0,Turkey,823225


In [23]:
#Q15. Find Country having lowest number of the death case
query = ''' SELECT "Country/Region", MIN(Deaths) AS Min_Deaths
FROM corona_data;
'''
death_cases = pd.read_sql(query, conn)
death_cases.head()

,Country/Region,Min_Deaths
0,Afghanistan,0


In [24]:
#Q16. Find top 5 countries having highest recovered case
query = ''' SELECT "Country/Region", MAX(Recovered) AS Max_Recovered
FROM corona_data
GROUP BY "Country/Region"
ORDER BY Max_Recovered DESC
LIMIT 5;
'''
top_5_countries = pd.read_sql(query, conn)
top_5_countries.head()


,Country/Region,Max_Recovered
0,Turkey,1123456
1,India,422436
2,Brazil,388340
3,US,150267
4,Colombia,89557
